## 加载模型

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")

In [ ]:
total_param = sum(param.numel() for param in model.parameters())
print(total_param)

## 准备数据集

In [3]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

In [ ]:
ds = Dataset.load_from_disk('./data')
ds[:3]

In [7]:
def process_func(example):
  MAX_LENGTH = 256
  instruction = tokenizer('\n'.join(["Human: " + example['instruction'], example['input']]).strip() + "\n\nAssistant: ")
  response = tokenizer(example['output'] + tokenizer.eos_token)
  input_ids = instruction['input_ids'] + response['input_ids']
  attention_mask = instruction['attention_mask'] + response['attention_mask']
  labels = [-100] * len(instruction['input_ids']) + response['input_ids']
  if len(input_ids) > MAX_LENGTH:
    input_ids = input_ids[:MAX_LENGTH]
    attention_mask = attention_mask[:MAX_LENGTH]
    labels = labels[:MAX_LENGTH]
  return {
      'input_ids': input_ids,
      'attention_mask': attention_mask,
      'labels': labels
  }

In [ ]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)

## Prompt Tuning

In [ ]:
from peft import PromptTuningConfig, get_peft_model, TaskType, PromptTuningInit
## config

### soft prompt
# confgi = PromptTuningConfig(
#     task_type=TaskType.CAUSAL_LM,
#     prompt_tuning_init=PromptTuningInit.RANDOM,
#     num_virtual_tokens=8,
#     tokenizer_name_or_path="Langboat/bloom-1b4-zh",
# )

### hard prompt
config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    prompt_tuning_init_text="下面是一段人与机器人的对话",
    num_virtual_tokens=len(tokenizer("下面是一段人与机器人的对话")["input_ids"]),
    tokenizer_name_or_path="Langboat/bloom-1b4-zh",
)
prompt_tuning_model = get_peft_model(model, config)
prompt_tuning_model.print_trainable_parameters()


## P-Tuning

In [ ]:
from peft import PromptEncoderConfig, TaskType, get_peft_model, PromptEncoderReparameterizationType
config = PromptEncoderConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10, 
                            encoder_reparameterization_type=PromptEncoderReparameterizationType.MLP,
                            encoder_num_layers=5, encoder_dropout=0.1, encoder_hidden_size=1024)

p_tuning_model = get_peft_model(model, config)
p_tuning_model.print_trainable_parameters()

## Prefix-Tuning

In [ ]:
from peft import PrefixTuningConfig, get_peft_model, TaskType
config = PrefixTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    num_virtual_tokens=10,
    prefix_projection=True,
)
prefix_tuning_model = get_peft_model(model, config)
prefix_tuning_model.print_trainable_parameters()

## LoRA

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=r".*query_key_value",
)
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

## IA3

In [ ]:
from peft import IA3Config
config = IA3Config(
    task_type=TaskType.CAUSAL_LM,
    target_modules="query_key_value",
    feedforward_modules=r"mlp.*"
)
IA3_model = get_peft_model(model, config)
IA3_model.print_trainable_parameters()

## 进阶用法
1. 自定义模型适配
    需要指定 `target_modules`参数

2.  一个主模型，多个适配器的情况如何使用
    1. 先使用PeftModel得到加载了lora adapter的model
    2. 然后可以使用 `load_adapter`方法进行加载其他 adapter
    3. 切换adapter可以使用 `set_adapter`方法进行激活adapter

3. 禁用适配器
    使用`with model.disable_adapter():`即可



## 训练

In [ ]:
import os
os.environ["WANDB_PROJECT"] = "llm-learning"

args = TrainingArguments(
    output_dir='./output/prompt_tuning',
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1,
    run_name="prompt_ft",
    save_steps=20
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)
trainer.train()

## 加载

In [ ]:
from peft import PeftModel
peft_model = PeftModel.from_pretrained(model, model_id=".output/...")
peft_model.eval()
inputs = tokenizer("Human: 提供几个有关数学考试的技巧。\n\nAssistant:", return_tensors="pt").to(peft_model.device)
print(tokenizer.decode(peft_model.generate(**inputs)[0], skip_special_tokens=True))